# Lesson:0 Quick Start Pytorch

### TorchVision  FashionMNIST datasets

### Importing dependencies

In [25]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## Download FashionMNIST datasets for training

In [26]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

### Download FashionMNIST datasets for testing

In [27]:
# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

### pass the Dataset as an argument to DataLoader

In [28]:
batch_size = 64

#Create data loaders.
train_dataloader =DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X[N,C,H,W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X[N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

In [29]:
# get cup or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

#Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
  
model = NeuralNetwork().to(device)
print(model)


Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

To train a model, we need a loss function and an optimzer.

In [30]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In a single training loop, the model makes predicitions on the training dataset(fed to it in batches), and backpropagates the predicition error to adjust the model's parameters.

In [31]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

We also check the model's performance againt the test dataset to ensure it is learning.

In [38]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations(epochs). During each epoch, the model learns parameters to make better predicitions. We print the model's accuracy and loss at each epoch; we'd like to see the accuracy increase and the loss decrease with every epoch.

In [39]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n..........................")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
..........................
loss: 0.751300 [   64/60000]
loss: 0.831254 [ 6464/60000]
loss: 0.594660 [12864/60000]
loss: 0.806192 [19264/60000]
loss: 0.727520 [25664/60000]
loss: 0.702643 [32064/60000]
loss: 0.785460 [38464/60000]
loss: 0.771666 [44864/60000]
loss: 0.774306 [51264/60000]
loss: 0.739570 [57664/60000]
Test Error: 
 Accuracy: 73.5%, Avg loss: 0.735092 

Epoch 2
..........................
loss: 0.718963 [   64/60000]
loss: 0.805360 [ 6464/60000]
loss: 0.568080 [12864/60000]
loss: 0.786164 [19264/60000]
loss: 0.708062 [25664/60000]
loss: 0.681991 [32064/60000]
loss: 0.763751 [38464/60000]
loss: 0.757085 [44864/60000]
loss: 0.755757 [51264/60000]
loss: 0.721649 [57664/60000]
Test Error: 
 Accuracy: 74.3%, Avg loss: 0.715709 

Epoch 3
..........................
loss: 0.690341 [   64/60000]
loss: 0.781673 [ 6464/60000]
loss: 0.544826 [12864/60000]
loss: 0.768617 [19264/60000]
loss: 0.690721 [25664/60000]
loss: 0.664434 [32064/60000]
loss: 0.743593 [38464/60000]
loss: 0.

# Saving Model

In [40]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Model

The process for loading a model include re-creating the model structure and loading the state dictionary into it.

In [41]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predicitons.

In [43]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

In [63]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
